# Example - generation with GUIDE and other models

In this notebook, we show how to use generation with GUIDE with other models such as Llama and Qwen

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
while "examples" in os.getcwd():
    os.chdir("..")

# from pipeline import 
from transformers import pipeline
from transformers.pipelines import PIPELINE_REGISTRY
from transformers import AutoModelForCausalLM
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig

from pipeline.pay_attention_pipeline import PayAttentionPipeline, AttentionLevels
import torch

import matplotlib.pyplot as plt

## Verifying that the pipeline is registered

In [3]:
PIPELINE_REGISTRY.check_task("pay-attention")

('pay-attention',
 {'impl': pipeline.pay_attention_pipeline.PayAttentionPipeline,
  'pt': (transformers.models.auto.modeling_auto.AutoModelForCausalLM,),
  'tf': (),
  'default': {'model': {'pt': ('mistralai/Mistral-7B-Instruct-v0.1',
     '1b62ab7')}},
  'type': 'text'},
 None)

## Loading generative model

We recommend using only Mistral or Llama models for now, due to specific implementation details in HuggingFace.

In [4]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)


tokenizer = AutoTokenizer.from_pretrained(
    "meta-llama/Meta-Llama-3-8B-Instruct",
    cache_dir = "/Data"    
)

base_model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Meta-Llama-3-8B-Instruct",
    quantization_config = quantization_config,
    device_map="auto",
    attn_implementation="eager",
    cache_dir = "/Data" 
)

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Loading pipeline

In [6]:
pipe= pipeline(
    "pay-attention", 
    model = base_model,
    tokenizer = tokenizer, 
    model_kwargs=dict(cache_dir = "/Data"),
)

## Prompt

The prompt was chosen to stay within the model's context length of 4096 tokens while being large enough to trigger potential hallucinations by the model.

In [4]:
# extract from Harry Potter and the Sorcerer's Stone's book
prompt = '''
Harry Potter and the Sorcerer's Stone
CHAPTER ONE
THE BOY WHO LIVED
Mr. and Mrs. Dursley, of number four, Privet Drive, were proud to say
that they were perfectly normal, thank you very much. They were the last
people you'd expect to be involved in anything strange or mysterious,
because they just didn't hold with such nonsense.
Mr. Dursley was the director of a firm called Grunnings, which made
drills. He was a big, beefy man with hardly any neck, although he did
have a very large mustache. Mrs. Dursley was thin and blonde and had
nearly twice the usual amount of neck, which came in very useful as she
spent so much of her time craning over garden fences, spying on the
neighbors. The Dursleys had a small son called Dudley and in their
opinion there was no finer boy anywhere.
The Dursleys had everything they wanted, but they also had a secret, and
their greatest fear was that somebody would discover it. They didn't
think they could bear it if anyone found out about the Potters. Mrs.
Potter was Mrs. Dursley's sister, but they hadn't met for several years;
in fact, Mrs. Dursley pretended she didn't have a sister, because her
sister and her good-for-nothing husband were as unDursleyish as it was
possible to be. The Dursleys shuddered to think what the neighbors would
say if the Potters arrived in the street. The Dursleys knew that the
Potters had a small son, too, but they had never even seen him. This boy
was another good reason for keeping the Potters away; they didn't want
Dudley mixing with a child like that.
When Mr. and Mrs. Dursley woke up on the dull, gray Tuesday our story
starts, there was nothing about the cloudy sky outside to suggest that
strange and mysterious things would soon be happening all over the
country. Mr. Dursley hummed as he picked out his most boring tie for
work, and Mrs. Dursley gossiped away happily as she wrestled a screaming
Dudley into his high chair.
None of them noticed a large, tawny owl flutter past the window.
At half past eight, Mr. Dursley picked up his briefcase, pecked Mrs.
Dursley on the cheek, and tried to kiss Dudley good-bye but missed,
2
because Dudley was now having a tantrum and throwing his cereal at the
walls. "Little tyke," chortled Mr. Dursley as he left the house. He got
into his car and backed out of number four's drive.
It was on the corner of the street that he noticed the first sign of
something peculiar -- a cat reading a map. For a second, Mr. Dursley
didn't realize what he had seen -- then he jerked his head around to
look again. There was a tabby cat standing on the corner of Privet
Drive, but there wasn't a map in sight. What could he have been thinking
of? It must have been a trick of the light. Mr. Dursley blinked and
stared at the cat. It stared back. As Mr. Dursley drove around the
corner and up the road, he watched the cat in his mirror. It was now
reading the sign that said Privet Drive -- no, looking at the sign; cats
couldn't read maps or signs. Mr. Dursley gave himself a little shake and
put the cat out of his mind. As he drove toward town he thought of
nothing except a large order of drills he was hoping to get that day.
But on the edge of town, drills were driven out of his mind by something
else. As he sat in the usual morning traffic jam, he couldn't help
noticing that there seemed to be a lot of strangely dressed people
about. People in cloaks. Mr. Dursley couldn't bear people who dressed in
funny clothes -- the getups you saw on young people! He supposed this
was some stupid new fashion. He drummed his fingers on the steering
wheel and his eyes fell on a huddle of these weirdos standing quite
close by. They were whispering excitedly together. Mr. Dursley was
enraged to see that a couple of them weren't young at all; why, that man
had to be older than he was, and wearing an emerald-green cloak! The
nerve of him! But then it struck Mr. Dursley that this was probably some
silly stunt -- these people were obviously collecting for something...
yes, that would be it. The traffic moved on and a few minutes later, Mr.
Dursley arrived in the Grunnings parking lot, his mind back on drills.
Mr. Dursley always sat with his back to the window in his office on the
ninth floor. If he hadn't, he might have found it harder to concentrate
on drills that morning. He didn't see the owls swoop ing past in broad
daylight, though people down in the street did; they pointed and gazed
open- mouthed as owl after owl sped overhead. Most of them had never
seen an owl even at nighttime. Mr. Dursley, however, had a perfectly
normal, owl-free morning. He yelled at five different people. He made
several important telephone calls and shouted a bit more. He was in a
very good mood until lunchtime, when he thought he'd stretch his legs
and walk across the road to buy himself a bun from the bakery.
3
He'd forgotten all about the people in cloaks until he passed a group of
them next to the baker's. He eyed them angrily as he passed. He didn't
know why, but they made him uneasy. This bunch were whispering
excitedly, too, and he couldn't see a single collecting tin. It was on
his way back past them, clutching a large doughnut in a bag, that he
caught a few words of what they were saying.
"The Potters, that's right, that's what I heard yes, their son, Harry"
Mr. Dursley stopped dead. Fear flooded him. He looked back at the
whisperers as if he wanted to say something to them, but thought better
of it.
He dashed back across the road, hurried up to his office, snapped at his
secretary not to disturb him, seized his telephone, and had almost
finished dialing his home number when he changed his mind. He put the
receiver back down and stroked his mustache, thinking... no, he was
being stupid. Potter wasn't such an unusual name. He was sure there were
lots of people called Potter who had a son called Harry. Come to think
of it, he wasn't even sure his nephew was called Harry. He'd never even
seen the boy. It might have been Harvey. Or Harold. There was no point
in worrying Mrs. Dursley; she always got so upset at any mention of her
sister. He didn't blame her -- if he'd had a sister like that... but all
the same, those people in cloaks...
He found it a lot harder to concentrate on drills that afternoon and
when he left the building at five o'clock, he was still so worried that
he walked straight into someone just outside the door.
"Sorry," he grunted, as the tiny old man stumbled and almost fell. It
was a few seconds before Mr. Dursley realized that the man was wearing a
violet cloak. He didn't seem at all upset at being almost knocked to the
ground. On the contrary, his face split into a wide smile and he said in
a squeaky voice that made passersby stare, "Don't be sorry, my dear sir,
for nothing could upset me today! Rejoice, for You-Know-Who has gone at
last! Even Muggles like yourself should be celebrating, this happy,
happy day!"
And the old man hugged Mr. Dursley around the middle and walked off.
Mr. Dursley stood rooted to the spot. He had been hugged by a complete
stranger. He also thought he had been called a Muggle, whatever that
was. He was rattled. He hurried to his car and set off for home, hoping
4
he was imagining things, which he had never hoped before, because he
didn't approve of imagination.
As he pulled into the driveway of number four, the first thing he saw --
and it didn't improve his mood -- was the tabby cat he'd spotted that
morning. It was now sitting on his garden wall. He was sure it was the
same one; it had the same markings around its eyes.
"Shoo!" said Mr. Dursley loudly. The cat didn't move. It just gave him a
stern look. Was this normal cat behavior? Mr. Dursley wondered. Trying
to pull himself together, he let himself into the house. He was still
determined not to mention anything to his wife.
Mrs. Dursley had had a nice, normal day. She told him over dinner all
about Mrs. Next Door's problems with her daughter and how Dudley had
learned a new word ("Won't!"). Mr. Dursley tried to act normally. When
Dudley had been put to bed, he went into the living room in time to
catch the last report on the evening news:
"And finally, bird-watchers everywhere have reported that the nation's
owls have been behaving very unusually today. Although owls normally
hunt at night and are hardly ever seen in daylight, there have been
hundreds of sightings of these birds flying in every direction since
sunrise. Experts are unable to explain why the owls have suddenly
changed their sleeping pattern." The newscaster allowed himself a grin.
"Most mysterious. And now, over to Jim McGuffin with the weather. Going
to be any more showers of owls tonight, Jim?"
"Well, Ted," said the weatherman, "I don't know about that, but it's not
only the owls that have been acting oddly today. Viewers as far apart as
Kent, Yorkshire, and Dundee have been phoning in to tell me that instead
of the rain I promised yesterday, they've had a downpour of shooting
stars! Perhaps people have been celebrating Bonfire Night early -- it's
not until next week, folks! But I can promise a wet night tonight."
Mr. Dursley sat frozen in his armchair. Shooting stars all over Britain?
Owls flying by daylight? Mysterious people in cloaks all over the place?
And a whisper, a whisper about the Potters...
Mrs. Dursley came into the living room carrying two cups of tea. It was
no good. He'd have to say something to her. He cleared his throat
nervously. "Er -- Petunia, dear -- you haven't heard from your sister
lately, have you?"
5
As he had expected, Mrs. Dursley looked shocked and angry. After all,
they normally pretended she didn't have a sister.
"No," she said sharply. "Why?"
"Funny stuff on the news," Mr. Dursley mumbled. "Owls... shooting
stars... and there were a lot of funny-looking people in town today..."
"So?" snapped Mrs. Dursley.
"Well, I just thought... maybe... it was something to do with... you
know... her crowd."
Mrs. Dursley sipped her tea through pursed lips. Mr. Dursley wondered
whether he dared tell her he'd heard the name "Potter." He decided he
didn't dare. Instead he said, as casually as he could, "Their son --
he'd be about Dudley's age now, wouldn't he?"
"I suppose so," said Mrs. Dursley stiffly.
"What's his name again? Howard, isn't it?"
"Harry. Nasty, common name, if you ask me."
"Oh, yes," said Mr. Dursley, his heart sinking horribly. "Yes, I quite
agree."
He didn't say another word on the subject as they went upstairs to bed.
While Mrs. Dursley was in the bathroom, Mr. Dursley crept to the bedroom
window and peered down into the front garden. The cat was still there.
It was staring down Privet Drive as though it were waiting for
something.
Was he imagining things? Could all this have anything to do with the
Potters? If it did... if it got out that they were related to a pair of
-- well, he didn't think he could bear it.
The Dursleys got into bed. Mrs. Dursley fell asleep quickly but Mr.
Dursley lay awake, turning it all over in his mind. His last, comforting
thought before he fell asleep was that even if the Potters were
involved, there was no reason for them to come near him and Mrs.
Dursley. The Potters knew very well what he and Petunia thought about
6
them and their kind.... He couldn't see how he and Petunia could get
mixed up in anything that might be going on -- he yawned and turned over
-- it couldn't affect them....
How very wrong he was.
Mr. Dursley might have been drifting into an uneasy sleep, but the cat
on the wall outside was showing no sign of sleepiness. It was sitting as
still as a statue, its eyes fixed unblinkingly on the far corner of
Privet Drive. It didn't so much as quiver when a car door slammed on the
next street, nor when two owls swooped overhead. In fact, it was nearly
midnight before the cat moved at all.
A man appeared on the corner the cat had been watching, appeared so
suddenly and silently you'd have thought he'd just popped out of the
ground. The cat's tail twitched and its eyes narrowed.
Nothing like this man had ever been seen on Privet Drive. He was tall,
thin, and very old, judging by the silver of his hair and beard, which
were both long enough to tuck into his belt. He was wearing long robes,
a purple cloak that swept the ground, and high-heeled, buckled boots.
His blue eyes were light, bright, and sparkling behind half-moon
spectacles and his nose was very long and crooked, as though it had been
broken at least twice. This man's name was Albus Dumbledore.
Albus Dumbledore didn't seem to realize that he had just arrived in a
street where everything from his name to his boots was unwelcome. He was
busy rummaging in his cloak, looking for something. But he did seem to
realize he was being watched, because he looked up suddenly at the cat,
which was still staring at him from the other end of the street. For
some reason, the sight of the cat seemed to amuse him. He chuckled and
muttered, "I should have known."
He found what he was looking for in his inside pocket. It seemed to be a
silver cigarette lighter. He flicked it open, held it up in the air, and
clicked it. The nearest street lamp went out with a little pop. He
clicked it again -- the next lamp flickered into darkness. Twelve times
he clicked the Put-Outer, until the only lights left on the whole street
were two tiny pinpricks in the distance, which were the eyes of the cat
watching him. If anyone looked out of their window now, even beady-eyed
Mrs. Dursley, they wouldn't be able to see anything that was happening
down on the pavement. Dumbledore slipped the Put-Outer back inside his
cloak and set off down the street toward number four, where he sat down
7
on the wall next to the cat. He didn't look at it, but after a moment he
spoke to it.
"Fancy seeing you here, Professor McGonagall."
He turned to smile at the tabby, but it had gone. Instead he was smiling
at a rather severe-looking woman who was wearing square glasses exactly
the shape of the markings the cat had had around its eyes. She, too, was
wearing a cloak, an emerald one. Her black hair was drawn into a tight
bun. She looked distinctly ruffled.
"How did you know it was me?" she asked.
"My dear Professor, I 've never seen a cat sit so stiffly."
"You'd be stiff if you'd been sitting on a brick wall all day," said
Professor McGonagall.
"All day? When you could have been celebrating? I must have passed a
dozen feasts and parties on my way here."
Professor McGonagall sniffed angrily.

'''

In [32]:
context_length = len(tokenizer(prompt)['input_ids'])
context_length

3682

In [33]:

message = [{'role': "user", "content":"Summarize in French" + prompt}]
normal_output = pipe(message, max_new_tokens = 100)

In [34]:
normal_output[0]['generated_text']

[{'role': 'user',
  'content': 'Summarize in French\nHarry Potter and the Sorcerer\'s Stone\nCHAPTER ONE\nTHE BOY WHO LIVED\nMr. and Mrs. Dursley, of number four, Privet Drive, were proud to say\nthat they were perfectly normal, thank you very much. They were the last\npeople you\'d expect to be involved in anything strange or mysterious,\nbecause they just didn\'t hold with such nonsense.\nMr. Dursley was the director of a firm called Grunnings, which made\ndrills. He was a big, beefy man with hardly any neck, although he did\nhave a very large mustache. Mrs. Dursley was thin and blonde and had\nnearly twice the usual amount of neck, which came in very useful as she\nspent so much of her time craning over garden fences, spying on the\nneighbors. The Dursleys had a small son called Dudley and in their\nopinion there was no finer boy anywhere.\nThe Dursleys had everything they wanted, but they also had a secret, and\ntheir greatest fear was that somebody would discover it. They didn\'t\

Note that the output is in French. The behavior of Llama 8B is different from Mistral 7B here because Llama was trained on larger context length

Let's proceed with generation using GUIDE, applying all three levels of enhancement.

In [28]:
pipe= pipeline(
    "pay-attention", 
    model = base_model,
    tokenizer = tokenizer, 
    model_kwargs=dict(cache_dir = "/Data"),
    **dict(delta_mid = 0)
)

In [29]:
message_1 = [{'role': 'user', 'content': "<!-> Summarize in French <-!>" + prompt}]
out_1 = pipe(message_1, max_new_tokens = 100)

Applying level 1 enhancement to the prompt. Delta = 0.0
Inserting special attention on 'Summarize in French'


In [30]:
out_1

[{'generated_text': [{'role': 'user',
    'content': '<!-> Summarize in French <-!>\nHarry Potter and the Sorcerer\'s Stone\nCHAPTER ONE\nTHE BOY WHO LIVED\nMr. and Mrs. Dursley, of number four, Privet Drive, were proud to say\nthat they were perfectly normal, thank you very much. They were the last\npeople you\'d expect to be involved in anything strange or mysterious,\nbecause they just didn\'t hold with such nonsense.\nMr. Dursley was the director of a firm called Grunnings, which made\ndrills. He was a big, beefy man with hardly any neck, although he did\nhave a very large mustache. Mrs. Dursley was thin and blonde and had\nnearly twice the usual amount of neck, which came in very useful as she\nspent so much of her time craning over garden fences, spying on the\nneighbors. The Dursleys had a small son called Dudley and in their\nopinion there was no finer boy anywhere.\nThe Dursleys had everything they wanted, but they also had a secret, and\ntheir greatest fear was that somebody 

In [12]:
message_2 = [{'role': 'user', 'content': "<!!-> Summarize in French <-!!>" + prompt}]
out_2 = pipe(message_2, max_new_tokens = 100)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Applying level 2 enhancement to the prompt. Delta = 1.0
Inserting special attention on 'Summarize in French'


In [13]:
out_2

[{'generated_text': [{'role': 'user',
    'content': '<!!-> Summarize in French <-!!>\nHarry Potter and the Sorcerer\'s Stone\nCHAPTER ONE\nTHE BOY WHO LIVED\nMr. and Mrs. Dursley, of number four, Privet Drive, were proud to say\nthat they were perfectly normal, thank you very much. They were the last\npeople you\'d expect to be involved in anything strange or mysterious,\nbecause they just didn\'t hold with such nonsense.\nMr. Dursley was the director of a firm called Grunnings, which made\ndrills. He was a big, beefy man with hardly any neck, although he did\nhave a very large mustache. Mrs. Dursley was thin and blonde and had\nnearly twice the usual amount of neck, which came in very useful as she\nspent so much of her time craning over garden fences, spying on the\nneighbors. The Dursleys had a small son called Dudley and in their\nopinion there was no finer boy anywhere.\nThe Dursleys had everything they wanted, but they also had a secret, and\ntheir greatest fear was that somebod

In [14]:
message_3 = [{'role': 'user', 'content': "<!!!-> Summarize in French <-!!!>" + prompt}]
out_3 = pipe(message_3, max_new_tokens = 100)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Applying level 3 enhancement to the prompt. Delta = 2.0
Inserting special attention on 'Summarize in French'


In [15]:
out_3

[{'generated_text': [{'role': 'user',
    'content': '<!!!-> Summarize in French <-!!!>\nHarry Potter and the Sorcerer\'s Stone\nCHAPTER ONE\nTHE BOY WHO LIVED\nMr. and Mrs. Dursley, of number four, Privet Drive, were proud to say\nthat they were perfectly normal, thank you very much. They were the last\npeople you\'d expect to be involved in anything strange or mysterious,\nbecause they just didn\'t hold with such nonsense.\nMr. Dursley was the director of a firm called Grunnings, which made\ndrills. He was a big, beefy man with hardly any neck, although he did\nhave a very large mustache. Mrs. Dursley was thin and blonde and had\nnearly twice the usual amount of neck, which came in very useful as she\nspent so much of her time craning over garden fences, spying on the\nneighbors. The Dursleys had a small son called Dudley and in their\nopinion there was no finer boy anywhere.\nThe Dursleys had everything they wanted, but they also had a secret, and\ntheir greatest fear was that someb

## Trying with Gemma

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
while "examples" in os.getcwd():
    os.chdir("..")

# from pipeline import 
from transformers import pipeline
from transformers.pipelines import PIPELINE_REGISTRY
from transformers import AutoModelForCausalLM
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig

from pipeline.pay_attention_pipeline import PayAttentionPipeline, AttentionLevels
import torch

import matplotlib.pyplot as plt

In [3]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)


tokenizer = AutoTokenizer.from_pretrained(
    
    "google/gemma-2-2b-it",
    cache_dir = "/Data"    
)

base_model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2-2b-it",
    quantization_config = quantization_config,
    device_map="auto",
    attn_implementation="eager",
    cache_dir = "/Data" 
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
prompt= '''
Brazil,[b] officially the Federative Republic of Brazil,[c] is the largest and easternmost country in South America and Latin America. It is the world's fifth-largest country by area and the seventh most populous. Its capital is Brasília, and its most populous city is São Paulo. Brazil is a federation composed of 26 states and a Federal District. It is the only country in the Americas where Portuguese is an official language.[12][13] Brazil is among the world's most multicultural and ethnically diverse nations, due to over a century of mass immigration from around the world.[14]

Bounded by the Atlantic Ocean on the east, Brazil has a coastline of 7,491 kilometers (4,655 mi).[15] Covering roughly half of South America's land area, it borders all other countries and territories on the continent except Ecuador and Chile.[16] Brazil's Amazon basin includes a vast tropical forest home to diverse wildlife, a variety of ecological systems, and extensive natural resources spanning numerous protected habitats.[15] This unique environmental heritage positions Brazil at number one of 17 megadiverse countries. The country's natural richness is also the subject of significant global interest, as environmental degradation (through processes such as deforestation) has direct impacts on global issues such as climate change and biodiversity loss.

The territory of present-day Brazil was inhabited by numerous tribal nations prior to the landing of explorer Pedro Álvares Cabral in 1500. Subsequently claimed by the Portuguese Empire, Brazil remained a Portuguese colony until 1808, when the capital of the empire was transferred from Lisbon to Rio de Janeiro. In 1815, the colony was elevated to the rank of kingdom upon the formation of the United Kingdom of Portugal, Brazil and the Algarves. Independence was achieved in 1822 with the creation of the Empire of Brazil, a unitary state governed under a constitutional monarchy and a parliamentary system. The ratification of the first constitution in 1824 led to the formation of a bicameral legislature, now called the National Congress. Slavery was abolished in 1888. The country became a presidential republic in 1889 following a military coup d'état. An authoritarian military dictatorship emerged in 1964 and ruled until 1985, after which civilian governance resumed. Brazil's current constitution, formulated in 1988, defines it as a democratic federal republic.[17] Due to its rich culture and history, the country ranks thirteenth in the world by number of UNESCO World Heritage Sites.[18]

Brazil is a regional and middle power[19][20][21] that is an emerging power[22][23][24][25] and a major non-NATO ally of the United States.[26] Categorized as a developing country with a high Human Development Index,[27] Brazil is considered an advanced emerging economy,[28] having the eighth largest GDP in the world in both nominal and PPP terms—the largest in Latin America and the Southern Hemisphere.[8][29] Classified as an upper-middle income economy by the World Bank,[30] and a newly industrialized country by the IMF,[31] Brazil has the largest share of wealth and the most complex economy in South America. It is also one of the world's major breadbaskets, being the largest producer of coffee for the last 150 years.[32] Despite its growing economic and global profile, the country continues to face high levels of corruption, crime and social inequality. Brazil is a founding member of the United Nations, the G20, BRICS, G4, Mercosul, Organization of American States, Organization of Ibero-American States and the Community of Portuguese Language Countries. Brazil is also an observer state of the Arab League.[33]

Etymology
Main article: Name of Brazil
The word Brazil probably comes from the Portuguese word for brazilwood, a tree that once grew plentifully along the Brazilian coast.[34] In Portuguese, brazilwood is called pau-brasil, with the word brasil commonly given the etymology "red like an ember", formed from brasa ('ember') and the suffix -il (from -iculum or -ilium).[35] It has alternatively been suggested that this is a folk etymology for a word for the plant related to an Arabic or Asian word for a red plant.[36] As brazilwood produces a deep red dye, it was highly valued by the European textile industry and was the earliest commercially exploited product from Brazil.[37] Throughout the 16th century, massive amounts of brazilwood were harvested by indigenous peoples (mostly Tupi) along the Brazilian coast, who sold the timber to European traders (mostly Portuguese, but also French) in return for assorted European consumer goods.[38]

The official Portuguese name of the land, in original Portuguese records, was the "Land of the Holy Cross" (Terra da Santa Cruz),[39] but European sailors and merchants commonly called it the "Land of Brazil" (Terra do Brasil) because of the brazilwood trade.[40] The popular appellation eclipsed and eventually supplanted the official Portuguese name. Some early sailors called it the "Land of Parrots".[41]

In the Guaraní language, an official language of Paraguay, Brazil is called "Pindorama", meaning 'land of the palm trees'.[42]

History
Main article: History of Brazil
For a chronological guide, see Timeline of Brazilian history.
Pre-Cabraline era
Main article: Pre-Cabraline history of Brazil
See also: Indigenous peoples in Brazil and Marajoara culture

Rock art at Serra da Capivara National Park, one of the largest and oldest concentrations of prehistoric sites in the Americas[43]
Some of the earliest human remains found in the Americas, Luzia Woman, were found in the area of Pedro Leopoldo, Minas Gerais and provide evidence of human habitation going back at least 11,000 years.[44][45] The earliest pottery ever found in the Western Hemisphere was excavated in the Amazon basin of Brazil and radiocarbon dated to 8,000 years ago (6000 BC). The pottery was found near Santarém and provides evidence that the region supported a complex prehistoric culture.[46] The Marajoara culture flourished on Marajó in the Amazon delta from AD 400 to 1400, developing sophisticated pottery, social stratification, large populations, mound building, and complex social formations such as chiefdoms.[47]

Around the time of the Portuguese arrival, the territory of current day Brazil had an estimated indigenous population of 7 million people,[48] mostly semi-nomadic, who subsisted on hunting, fishing, gathering, and migrant agriculture. The population comprised several large indigenous ethnic groups (e.g., the Tupis, Guaranis, Gês, and Arawaks). The Tupi people were subdivided into the Tupiniquins and Tupinambás.[49]

Before the arrival of the Europeans, the boundaries between these groups and their subgroups were marked by wars that arose from differences in culture, language and moral beliefs.[50] These wars also involved large-scale military actions on land and water, with cannibalistic rituals on prisoners of war.[51][52] While heredity had some weight, leadership was a status more won over time than assigned in succession ceremonies and conventions.[50] Slavery among the indigenous groups had a different meaning than it had for Europeans, since it originated from a diverse socioeconomic organization, in which asymmetries were translated into kinship relations.[53]

Portuguese colonization
Main articles: Colonial Brazil and Portuguese Empire
See also: Slavery in Brazil, War of the Emboabas, and Minas Gerais Conspiracy

Pedro Álvares Cabral landing in Porto Seguro in 1500, ushering in more than 300 years of Portuguese rule

Ouro Preto, Minas Gerais, was the center of the Brazilian Gold Rush and was designated a World Heritage Site by UNESCO due to its Baroque colonial architecture.

Execution of the Punishment of the Whip by Jean-Baptiste Debret. Nearly 5 million enslaved Africans were imported to Brazil during the Atlantic slave trade, more than any country.[54]
Following the 1494 Treaty of Tordesillas, the land now called Brazil was claimed for the Portuguese Empire on 22 April 1500, with the arrival of the Portuguese fleet commanded by Pedro Álvares Cabral.[55] The Portuguese encountered indigenous peoples divided into several ethnic societies, most of whom spoke languages of the Tupi–Guarani family and fought among themselves.[56] Though the first settlement was founded in 1532, colonization effectively began in 1534, when King John III of Portugal divided the territory into the fifteen private and autonomous captaincies.[57][58]

However, the decentralized and unorganized tendencies of the captaincies proved problematic, and in 1549 the Portuguese king restructured them into the Governorate General of Brazil in the city of Salvador, which became the capital of a single and centralized Portuguese colony in South America.[58][59] In the first two centuries of colonization, Indigenous and European groups lived in constant war, establishing opportunistic alliances in order to gain advantages against each other.[60][61][62][63]

By the mid-16th century, cane sugar had become Brazil's most important export,[56][64] while slaves purchased in Sub-Saharan Africa in the slave market of Western Africa[65] (not only those from Portuguese allies of their colonies in Angola and Mozambique), had become its largest import,[66][67] to cope with sugarcane plantations, due to increasing international demand for Brazilian sugar.[68][69] Brazil received more than 2.8 million slaves from Africa between the years 1500 and 1800.[70]

By the end of the 17th century, sugarcane exports began to decline[71] and the discovery of gold by bandeirantes in the 1690s would become the new backbone of the colony's economy, fostering a gold rush[72] which attracted thousands of new settlers to Brazil from Portugal and all Portuguese colonies around the world.[73] This increased level of immigration in turn caused some conflicts between newcomers and old settlers.[74]

Portuguese expeditions known as bandeiras gradually expanded Brazil's original colonial frontiers in South America to its approximately current borders.[75][76] In this era other European powers tried to colonize parts of Brazil, in incursions that the Portuguese had to fight, notably the French in Rio during the 1560s, in Maranhão during the 1610s, and the Dutch in Bahia and Pernambuco, during the Dutch–Portuguese War, after the end of Iberian Union.[77]

The Portuguese colonial administration in Brazil had two objectives that would ensure colonial order and the monopoly of Portugal's wealthiest and largest colony: to keep under control and eradicate all forms of slave rebellion and resistance, such as the Quilombo of Palmares,[78] and to repress all movements for autonomy or independence, such as the Minas Gerais Conspiracy.[79]

Elevation to kingdom
Main article: United Kingdom of Portugal, Brazil and the Algarves
See also: Invasion of Portugal (1807) and Transfer of the Portuguese court to Brazil

The Acclamation of King João VI of the United Kingdom of Portugal, Brazil and the Algarves in Rio de Janeiro, 6 February 1818
In late 1807, Spanish and Napoleonic forces threatened the security of continental Portugal, causing Prince Regent John, in the name of Queen Maria I, to move the royal court from Lisbon to Rio de Janeiro.[80] There they established some of Brazil's first financial institutions, such as its local stock exchanges[81] and its National Bank, additionally ending the Portuguese monopoly on Brazilian trade and opening Brazil's ports to other nations. In 1809, in retaliation for being forced into exile, the Prince Regent ordered the conquest of French Guiana.[82]

With the end of the Peninsular War in 1814, the courts of Europe demanded that Queen Maria I and Prince Regent John return to Portugal, deeming it unfit for the head of an ancient European monarchy to reside in a colony. In 1815, to justify continuing to live in Brazil, where the royal court had thrived for six years, the Crown established the United Kingdom of Portugal, Brazil and the Algarves, thus creating a pluricontinental transatlantic monarchic state.[83] However, the leadership in Portugal, resentful of the new status of its larger colony, continued to demand the return of the court to Lisbon (see Liberal Revolution of 1820). In 1821, acceding to the demands of revolutionaries who had taken the city of Porto,[84] John VI departed for Lisbon. There he swore an oath to the new constitution, leaving his son, Prince Pedro de Alcântara, as Regent of the Kingdom of Brazil.[85]

Independent empire
Main articles: Independence of Brazil and Empire of Brazil

Declaration of the Brazilian independence by Pedro I on 7 September 1822
Tensions between Portuguese and Brazilians increased and the Portuguese Cortes, guided by the new political regime imposed by the Liberal Revolution, tried to re-establish Brazil as a colony.[86] The Brazilians refused to yield, and Prince Pedro decided to stand with them, declaring the country's independence from Portugal on 7 September 1822.[87] A month later, Prince Pedro was declared the first Emperor of Brazil, with the royal title of Dom Pedro I, resulting in the founding of the Empire of Brazil.[88]

The Brazilian War of Independence, which had already begun along this process, spread through the northern, northeastern regions and in the Cisplatina province.[89] The last Portuguese soldiers surrendered on 8 March 1824;[90] Portugal officially recognized Brazilian independence on 29 August 1825.[91]

On 7 April 1831, worn down by years of administrative turmoil and political dissent with both liberal and conservative sides of politics, including an attempt of republican secession[92] and unreconciled to the way that absolutists in Portugal had given in the succession of King John VI, Pedro I departed for Portugal to reclaim his daughter's crown after abdicating the Brazilian throne in favor of his five-year-old son and heir (Dom Pedro II).[93]


Pedro II, Emperor of Brazil between 1831 and 1889
As the new Emperor could not exert his constitutional powers until he came of age, a regency was set up by the National Assembly.[94] In the absence of a charismatic figure who could represent a moderate face of power, during this period a series of localized rebellions took place, such as the Cabanagem in Grão-Pará, the Malê Revolt in Salvador, the Balaiada (Maranhão), the Sabinada (Bahia), and the Ragamuffin War, which began in Rio Grande do Sul and was supported by Giuseppe Garibaldi. These emerged from the provinces' dissatisfaction with the central power, coupled with old and latent social tensions peculiar to a vast, slaveholding and newly independent nation state.[95] This period of internal political and social upheaval, which included the Praieira revolt in Pernambuco, was overcome only at the end of the 1840s, years after the end of the regency, which occurred with the premature coronation of Pedro II in 1841.[96]
'''

In [5]:
len(tokenizer(prompt)['input_ids'])

3328

In [6]:
pipe= pipeline(
    "pay-attention", 
    model = base_model,
    tokenizer = tokenizer, 
    model_kwargs=dict(cache_dir = "/Data"),
    # **dict(delta_mid = )
)

In [7]:
# pipe.guide_model.remove_hooks()

In [8]:
message = [{'role': "user", "content": "Summarize in French" + prompt}]
normal_output = pipe(message, max_new_tokens = 100)

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


In [52]:
normal_output

[{'generated_text': [{'role': 'user',
    'content': 'Summarize in French\nBrazil,[b] officially the Federative Republic of Brazil,[c] is the largest and easternmost country in South America and Latin America. It is the world\'s fifth-largest country by area and the seventh most populous. Its capital is Brasília, and its most populous city is São Paulo. Brazil is a federation composed of 26 states and a Federal District. It is the only country in the Americas where Portuguese is an official language.[12][13] Brazil is among the world\'s most multicultural and ethnically diverse nations, due to over a century of mass immigration from around the world.[14]\n\nBounded by the Atlantic Ocean on the east, Brazil has a coastline of 7,491 kilometers (4,655 mi).[15] Covering roughly half of South America\'s land area, it borders all other countries and territories on the continent except Ecuador and Chile.[16] Brazil\'s Amazon basin includes a vast tropical forest home to diverse wildlife, a va

In [9]:
message = [{'role': "user", "content":" <!!-> Summarize in French: <-!!>"  + prompt}]
out1 = pipe(message, max_new_tokens = 100, do_sample = True)

Applying level 2 enhancement to the prompt. Delta = 1.0
Inserting special attention on 'Summarize in French:'


In [10]:
out1

[{'generated_text': [{'role': 'user',
    'content': ' <!!-> Summarize in French: <-!!>\nBrazil,[b] officially the Federative Republic of Brazil,[c] is the largest and easternmost country in South America and Latin America. It is the world\'s fifth-largest country by area and the seventh most populous. Its capital is Brasília, and its most populous city is São Paulo. Brazil is a federation composed of 26 states and a Federal District. It is the only country in the Americas where Portuguese is an official language.[12][13] Brazil is among the world\'s most multicultural and ethnically diverse nations, due to over a century of mass immigration from around the world.[14]\n\nBounded by the Atlantic Ocean on the east, Brazil has a coastline of 7,491 kilometers (4,655 mi).[15] Covering roughly half of South America\'s land area, it borders all other countries and territories on the continent except Ecuador and Chile.[16] Brazil\'s Amazon basin includes a vast tropical forest home to diverse 